#Deployment do modelo de regressão de previsão de consumo de combustível (Tensorflow).
Utilizaremos o modelo criado através do [tutorial](https://www.tensorflow.org/tutorials/keras/regression?hl=pt-br) e persistido com a classe `save_model` (instruções baseadas no tutorial podem ser obtidas no [notebook](https://github.com/lfbraz/azure-databricks/blob/master/notebooks/tensorflow-regression-model.ipynb) *baseado no tutorial*)

##Criar/Utilizar Workspace do Azure Machine Learning

In [3]:
from azureml.core import Workspace, Dataset

WORKSPACE_NAME = '<NOME_WORKSPACE>'
WORKSPACE_LOCATION = '<REGIAO_WORKSPACE>'
RESOURCE_GROUP = '<NOME_RESOURCE_GROUP>'
SUBSCRIPTION_ID = '<ID_SUBSCRICAO>'

NOME_IMAGEM = 'model-image-linear-regression'
DESCRICAO_IMAGEM = 'Modelo de Regressão Linear - Tutorial'

workspace = Workspace.create(name = WORKSPACE_NAME,
                             location = WORKSPACE_LOCATION,
                             resource_group = RESOURCE_GROUP,
                             subscription_id = SUBSCRIPTION_ID,
                             exist_ok=True)


Por padrão será solicitada a autenticação utilizando o `Interactive Login`. Para cenários produtivos deve-se utilizar um registro de aplicação com `Service Principal`.

Na [documentação](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication#set-up-service-principal-authentication) temos uma explicação mais detalhada dos diferentes tipos de autenticação.

##Criar um entry point

O `entry script` somente possui duas funções obrigatórias, o `init()` e o `run()`. Estas funções são utilizadas para iniciar o serviço e executar o modelo utilizando os dados requisitados pelo cliente. Outras funções que podem ser adicionadas são relacionadas ao `loading` ou aplicação de tratamentos necessários para o `input`.

In [6]:
%%writefile /dbfs/models/model-regressao-tensorflow/score.py

import tensorflow as tf
from tensorflow.keras.models import load_model
import json
import pandas as pd
import os

# Called when the deployed service starts
def init():
    global model
    global train_stats

    # Get the path where the deployed model can be found.
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), './models/model-regressao-tensorflow')
    
    # Load keras
    model = load_model(model_path)
    
    # Load train_stats
    train_stats = pd.read_pickle(model_path + "/train_stats.pkl")

def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

# Handle requests to the service
def run(data):
  # JSON request.
  # {"Cylinders":0, "Displacement":0.0, "Horsepower":0.0, "Weight":0.0, "Acceleration":0.5, "Model Year":0, "USA":0.0, "Europe":0.0, "Japan":0.0}
  data = pd.DataFrame([json.loads(data)])

  # Apply norm function
  data = norm(data)

  # Return the prediction
  prediction = predict(data)
  
  return prediction

def predict(data):
  score = model.predict(data)[0][0]
  return {"MPG_PREDICAO": float(score)}

Overwriting /dbfs/models/model-regressao-tensorflow/score.py

##Definir configurações para deploy
Aqui temos que adicionar todos os pacotes necessários para predição do modelo. No caso deste exemplo precisamos do `tensorflow`, `pandas` e `azureml-sdk`.

In [8]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
env = Environment(name="tensorflow_env")

conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
conda_dep.add_conda_package("tensorflow")

# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("keras")
conda_dep.add_pip_package("pandas")

# Adds dependencies to PythonSection of myenv
env.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="/dbfs/models/model-regressao-tensorflow/score.py",
                                   environment=env)

##Registrar uma imagem com o modelo
Com as configurações definidas, podemos agora registrar o modelo no **Azure Machine Learning**. Será gerado uma nova versão do modelo a cada novo registro, em que através da interface do AML podemos ver os artefatos (modelos, etc.) atrelados ao registro de modelo gerado.

In [10]:
from azureml.core.model import Model

MODEL_NAME = 'model-regressao-tensorflow'
DESCRIPTION = 'Modelo de regressão utilizando tensorflow (keras)'
MODEL_PATH = '/dbfs/models'

model_azure = Model.register(model_path = MODEL_PATH,
                             model_name = MODEL_NAME,
                             description = DESCRIPTION,
                             workspace = workspace)

Registering model model-regressao-tensorflow

##Deploy
Agora com a imagem criada, podemos escolher dois tipos de deployment, utilizando ACI (Azure Container Image) ou AKS (Azure Kubernetes Service).

Para cenários de desenvolvimento é indicado o uso do ACI, já para cenários produtivos AKS terá melhores opções quanto a segurança e escalabilidade.

Neste exemplo mostraremos como realizar o `deployment` utilizando [ACI](https://azure.microsoft.com/en-us/services/container-instances/) com um container com 1 CPU core e 1GB de memória RAM.

In [12]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.exceptions import WebserviceException
from azureml.core.model import Model

ENDPOINT_NAME = 'car-regression-service'

# Remove any existing service under the same name.
try:
    Webservice(workspace, ENDPOINT_NAME).delete()
except WebserviceException:
    pass

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(workspace, ENDPOINT_NAME, [model_azure], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

print('A API {} foi gerada no estado {}'.format(service.scoring_uri, service.state))

Running...........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
A API http://4422c520-9906-42aa-843c-93744915d897.centralus.azurecontainer.io/score foi gerada no estado Healthy

##Chamada da API
Finalmente faremos o request da API utilizando a variável query_input. A URL da API pode ser obtida através do dev_webservice.scoring_uri que foi gerado no deploy do endpoint.

In [14]:
import requests
import json
import numpy as np
import pandas as pd

scoring_uri = service.scoring_uri
headers = {'Content-Type':'application/json'}

json_test = {"Cylinders":8, "Displacement":500, "Horsepower":200, "Weight":3850, "Acceleration":8, "Model Year":70, "USA":1, "Europe":0, "Japan":0}
json_test = json.dumps(json_test)

response = requests.post(scoring_uri, data=json_test, headers=headers)

print(response.status_code)
print(response.content)

200
b'{"MPG_PREDICAO": 15.83124828338623}'